In [1]:
import os
import numpy as np
import numpy.matlib
from scipy.io import wavfile
from pydub import AudioSegment

In [2]:
files = os.listdir("/home/andrea/Scrivania/MIVIA_ROAD_DB1/splitted_dataset/")
for i,f in enumerate(files):
    files[i] = "/home/andrea/Scrivania/MIVIA_ROAD_DB1/splitted_dataset/"+f
files = sorted(files)

In [20]:
maxlen = 0
for f in files:
    s = AudioSegment.from_file(f)
    samples = s.get_array_of_samples()
    if len(samples) > maxlen:
        maxlen = len(samples)

In [34]:
def fill(file, maxlength=264256):
    if len(file) < maxlength:
        complete = np.zeros((maxlength,))
        filled = 0
        while filled < maxlength:
            if filled+len(file) < maxlength:
                complete[filled:filled+len(file)] = file
                filled+=len(file)
            else:
                complete[filled:] = file[:maxlength-filled]
        return  complete
    return file

In [ ]:
noises = []
def create_random_bg_noises():
    import random
    for _ in range(15):
        audios = []
        for _ in range(3):
            chosen = None
            num = str(random.randint(400, len(files)))
            while chosen is None:
                chosen = searchFile(num)
            audios.append(chosen)
        base = AudioSegment.from_file(audios[0])
        base2 = base.overlay(AudioSegment.from_file(audios[1]))
        base3 = base2.overlay(AudioSegment.from_file(audios[2]))
        complete = fill(np.asarray(base3.get_array_of_samples()))
        noises.append(AudioSegment(complete.tobytes(),
                      frame_rate=base.frame_rate,
                      sample_width=base.frame_width,
                      channels=1))
create_random_bg_noises()

In [30]:
def add_noise_and_normalize_length(file, noise, maxlength=264256):
    new_audio = np.zeros((maxlength,))
    stest = AudioSegment.from_file(file)
    samples = np.asarray(stest.get_array_of_samples())
    new_audio[:maxlength] = samples
    s2 =  AudioSegment(new_audio.tobytes(),
                      frame_rate=stest.frame_rate,
                      sample_width=stest.frame_width,
                      channels=1)
    return s2.overlay(noise)

In [31]:
stest == s2


True

In [9]:

def searchFile(name):
    for f in files:
        splitted = os.path.split(f)
        num, ext = splitted[-1].split(".")
        if num == name:
            return f

In [7]:
def mix(audio1, audio2):
    sound1 = AudioSegment.from_file(audio1)
    sound2 = AudioSegment.from_file(audio2)
    return sound1.overlay(sound2)

In [10]:
s1 = AudioSegment.from_file(files[0])
s2 = AudioSegment.from_file(files[1])
s3 = mix(searchFile("0"), searchFile("1"))

In [5]:
files[730]

'/home/andrea/Scrivania/MIVIA_ROAD_DB1/splitted_dataset/89.wav'

In [ ]:
np_files = np.ndarray((len(files),))

In [14]:
events, bg = [],[]
np_classes = np.load("/home/andrea/Scrivania/MIVIA_ROAD_DB1/classes.npy")

for n in range(400):
    events.append(searchFile(str(n)))
for m in range(400,742):
    bg.append(searchFile(str(m)))

In [21]:
classes = []
counter = 0
for i, e in enumerate(events):
    for j, b in enumerate(bg):
        classes.append(np_classes[i])
        new = mix(e,b)
        new.export("/mnt/3206BFFF191E7F85/extended_dataset/"+str(counter)+".wav", format='wav')
        counter+=1
for i, e in enumerate(bg):
    for j, b in enumerate(bg):
        classes.append(0)
        new = mix(e,b)
        new.export("/mnt/3206BFFF191E7F85/extended_dataset/"+str(counter)+".wav", format='wav')
        counter+=1
classes_np = np.asarray(classes)

In [20]:
np.save("/home/andrea/Scrivania/MIVIA_ROAD_DB1/splitted_dataset/classes_ext.npy", classes_np)